# Библиотека Pandas
Когда и зачем использовать

Режим презентации jupyter nbconvert my_script.ipynb --to slides --post serve

https://medium.com/@mjspeck/presenting-code-using-jupyter-notebook-slides-a8a3c3b59d67

In [1]:
# запуск презентации
!D:/Users/max/anaconda3/bin/jupyter nbconvert Python_10_pandas_presentation_mode.ipynb --to slides --post serve

The system cannot find the path specified.


In [2]:
import pandas as pd

In [3]:
 import requests

#['ml-25m/', 'ml-25m/tags.csv', 'ml-25m/links.csv', 'ml-25m/README.txt', 'ml-25m/ratings.csv', 'ml-25m/genome-tags.csv', 'ml-25m/genome-scores.csv', 'ml-25m/movies.csv']
url = 'https://files.grouplens.org/datasets/movielens/ml-25m.zip'
file_to_unzip = 'ml-25m/ratings.csv'
 
content = requests.get(url)

# unzip the content
from io import BytesIO
from zipfile import ZipFile

f = ZipFile(BytesIO(content.content))  # zipfile.ZipFile
# print(f.namelist())

In [4]:
 with f.open('ml-25m/ratings.csv', 'r') as unzipped_file:    #  zipfile.ZipExtFile 
    ratings = pd.read_csv(unzipped_file)
   #  f.close()  # не закрываю, тк ниже еще буду читать

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
ratings.info

<bound method DataFrame.info of           userId  movieId  rating   timestamp
0              1      296     5.0  1147880044
1              1      306     3.5  1147868817
2              1      307     5.0  1147868828
3              1      665     5.0  1147878820
4              1      899     3.5  1147868510
...          ...      ...     ...         ...
25000090  162541    50872     4.5  1240953372
25000091  162541    55768     2.5  1240951998
25000092  162541    56176     2.0  1240950697
25000093  162541    58559     4.0  1240953434
25000094  162541    63876     5.0  1240952515

[25000095 rows x 4 columns]>

In [7]:
print ("Column".ljust(10), "Number of unique:".ljust(20), "Sum (isnull)" )
for c in ratings.columns:
    print (c.ljust(15), str(ratings[ c ].nunique()).ljust(20), ratings[ c ].isnull().sum() )

Column     Number of unique:    Sum (isnull)
userId          162541               0
movieId         59047                0
rating          10                   0
timestamp       20115267             0


странно, что в timestamp  всего 20 млн. уникальных записей, значит около 5млн выставляли оценки одновременно с точностью до секунды 

In [8]:
# сколько фильмов в коллекции
len(ratings.movieId.unique())

59047

In [9]:
%%time

# топ пользователей по количеству оценок
ratings.groupby('userId').count().sort_values('movieId', ascending=False).head()

CPU times: total: 734 ms
Wall time: 730 ms


,movieId,rating,timestamp
userId,,,
72315,32202,32202,32202
80974,9178,9178,9178
137293,8913,8913,8913
33844,7919,7919,7919
20055,7488,7488,7488


# Lifetime

In [10]:
ratings_grouped = ratings.groupby('userId').agg([min, max])
ratings_grouped.head()

movieId         rating        timestamp            
           min     max    min  max         min         max
userId                                                    
1          296   32591    0.5  5.0  1147868053  1147880055
2            1   40815    0.5  5.0  1141415509  1141418008
3            1  187593    2.0  5.0  1439472199  1566091883
4            1  203519    0.5  5.0  1573937091  1574195101
5            1    1409    2.0  5.0   830786155   858628421

In [11]:
ratings_grouped['diff'] = ratings_grouped['timestamp']['max'] - ratings_grouped['timestamp']['min']
ratings_grouped.head()

movieId         rating        timestamp                   diff
           min     max    min  max         min         max           
userId                                                               
1          296   32591    0.5  5.0  1147868053  1147880055      12002
2            1   40815    0.5  5.0  1141415509  1141418008       2499
3            1  187593    2.0  5.0  1439472199  1566091883  126619684
4            1  203519    0.5  5.0  1573937091  1574195101     258010
5            1    1409    2.0  5.0   830786155   858628421   27842266

In [12]:
# дней на сервисе
ratings_grouped['diff'].mean() / 24 / 3600

221.41065851262715

In [13]:
%%time

ratings_grouped = ratings.groupby('userId').agg([min, max])
ratings_grouped['diff'] = ratings_grouped['timestamp']['max'] - ratings_grouped['timestamp']['min']

ratings_grouped['diff'].mean() / 24 / 3600

CPU times: total: 1.36 s
Wall time: 1.37 s


221.41065851262715

# JOIN как в SQL

In [14]:
with f.open('ml-25m/movies.csv', 'r') as unzipped_file:    #  zipfile.ZipExtFile 
   movies = pd.read_csv(unzipped_file)
   f.close()
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
joined = ratings.merge(movies, on='movieId', how='left')
joined

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance
...,...,...,...,...,...,...
25000090,162541,50872,4.5,1240953372,Ratatouille (2007),Animation|Children|Drama
25000091,162541,55768,2.5,1240951998,Bee Movie (2007),Animation|Comedy
25000092,162541,56176,2.0,1240950697,Alvin and the Chipmunks (2007),Children|Comedy
25000093,162541,58559,4.0,1240953434,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX


In [16]:
joined.groupby('title').count().sort_values('rating', ascending=False).head()
# joined.head()

,userId,movieId,rating,timestamp,genres
title,,,,,
Forrest Gump (1994),81491,81491,81491,81491,81491
"Shawshank Redemption, The (1994)",81482,81482,81482,81482,81482
Pulp Fiction (1994),79672,79672,79672,79672,79672
"Silence of the Lambs, The (1991)",74127,74127,74127,74127,74127
"Matrix, The (1999)",72674,72674,72674,72674,72674
